<a href="https://colab.research.google.com/github/younesabdolmalaky/LTR-on-torob-data/blob/main/notebooks/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)
aggregated_search_data_path = '/content/drive/MyDrive/torob/output_data/aggregated_search_data.jsonl'
preprocessed_products_path = '/content/drive/MyDrive/torob/output_data/preprocessed_products.jsonl'
preprocessed_test_queries_path = '/content/drive/MyDrive/torob/output_data/preprocessed_test_queries.jsonl'

In [4]:
query_normal = pd.DataFrame(read_json_lines('/content/drive/MyDrive/torob/data/test-offline-data_v1.jsonl'))

In [5]:
products_data_df = pd.DataFrame(read_json_lines(preprocessed_products_path))

In [6]:
products_id_to_idx = dict(
    (p_id, idx)
    for idx, p_id in enumerate(products_data_df['id'])
)

In [7]:
doc = pickle.load(open('/content/drive/MyDrive/torob/Features/doc_tfidf.h5', "rb"))
query = pickle.load(open('/content/drive/MyDrive/torob/Features/test_query_tfidf.h5', "rb"))

In [8]:
rand = np.load('/content/drive/MyDrive/torob/Features/rand.npy')

In [9]:
def batch_generator(query_results , batch_size , query ,doc , products_id_to_idx):
  count = 0
  a = []
  b = []
  for qid, agg_search in (enumerate(query_results.itertuples(index=False))):
    for product_id in agg_search.result_not_ranked:
      if product_id is None:
            continue
      p_idx = products_id_to_idx[product_id]
      a.append(query[qid].dot(rand))
      b.append(doc[p_idx].dot(rand))
      count = count  + 1
      if(count>=batch_size):
        count = 0
        yield a,b
        a = []
        b = []

In [10]:
batch_size = 8196

In [18]:
counter = 0
for i in batch_generator(query_normal , batch_size ,query ,doc , products_id_to_idx):
  counter = counter + len(i[0])

In [18]:
counter = 1344144

In [13]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras import utils

In [14]:
def ranknet_loss(y_true, y_pred):
    y_true = 2 * y_true - 1

    pairwise_diff = tf.expand_dims(y_true, axis=1) - tf.expand_dims(y_true, axis=0)

    pairwise_logits = tf.expand_dims(y_pred, axis=1) - tf.expand_dims(y_pred, axis=0)

    mask = tf.cast(tf.not_equal(pairwise_diff, 0), tf.float32)
    pairwise_logits = pairwise_logits * mask

    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.sigmoid(pairwise_diff), logits=pairwise_logits))

    return loss

In [15]:
utils.get_custom_objects()['ranknet_loss'] = ranknet_loss

In [16]:
model = load_model('/content/drive/MyDrive/torob/models/model_1.h5')

In [20]:
arr = np.ones((counter))
batch = 0
coun = 0 
for i in batch_generator(query_normal , batch_size ,query ,doc , products_id_to_idx):
  test = i
  temp = np.hstack((np.array(i[0]).reshape(8196, 256) ,  np.array(i[1]).reshape(8196, 256)))
  arr[batch:batch+batch_size] = model.predict(temp).flatten()
  batch = batch + len(i[0])
  print(coun)
  coun = coun + 1

257/257 [==============================] - 3s 10ms/step
0
257/257 [==============================] - 4s 16ms/step
1
257/257 [==============================] - 3s 10ms/step
2
257/257 [==============================] - 3s 13ms/step
3
257/257 [==============================] - 3s 10ms/step
4
257/257 [==============================] - 3s 10ms/step
5
257/257 [==============================] - 4s 14ms/step
6
257/257 [==============================] - 3s 12ms/step
7
257/257 [==============================] - 3s 12ms/step
8
257/257 [==============================] - 3s 10ms/step
9
257/257 [==============================] - 3s 13ms/step
10
257/257 [==============================] - 3s 10ms/step
11
257/257 [==============================] - 4s 15ms/step
12
257/257 [==============================] - 2s 9ms/step
13
257/257 [==============================] - 3s 10ms/step
14
257/257 [==============================] - 4s 17ms/step
15
257/257 [==============================] - 3s 10ms/step
16
257/257 

In [38]:
a1 = []
b1 = []

In [39]:
count = 0

In [40]:
for qid, agg_search in (enumerate(query_normal.itertuples(index=False))):
  for product_id in agg_search.result_not_ranked:
    count = count  + 1
    if count < 1344144:
      continue
    if product_id is None:
          continue
    p_idx = products_id_to_idx[product_id]
    a1.append(query[qid].dot(rand))
    b1.append(doc[p_idx].dot(rand))


In [41]:
np.array(b1).shape

(5350, 1, 256)

In [42]:
temp = np.hstack((np.array(b1).reshape(5350,256 ) ,  np.array(b1).reshape(5350, 256)))

In [43]:
temp.shape

(5350, 512)

In [44]:
part1_pred = model.predict(temp).flatten()

168/168 [==============================] - 2s 12ms/step


In [48]:
pred = np.concatenate((arr , part1_pred))

In [49]:
test_predictions = []

In [51]:
test_data_df = pd.DataFrame(read_json_lines('/content/drive/MyDrive/torob/data/test-offline-data_v1.jsonl'))

In [52]:
start_idx = 0
for test_candidates in  test_data_df['result_not_ranked']:
  preds_sample = pred[start_idx:start_idx + len(test_candidates)]
  sorted_idx = np.argsort(preds_sample)[::-1]
  sorted_candidates = [test_candidates[i] for i in sorted_idx]
  test_predictions.append(sorted_candidates)
  start_idx += len(test_candidates)

In [53]:
def write_test_predictions(predictions_path, predictions):
    lines = []
    for preds in predictions:
        lines.append(",".join([str(p_id) for p_id in preds]))

    with open(predictions_path, 'w') as f:
        f.write("\n".join(lines))

In [54]:
write_test_predictions('/content/predictions.txt', test_predictions)